In [1]:
import numpy as np
import pandas as pd
import evaluate
import torch
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import CrossEntropyLoss

from preprocessing import climate_fever_to_claim_evidence_pairs

In [ ]:
# This should always output true now, but worth checking
print(f"GPU available: {torch.cuda.is_available()}")

is GPU available: True


In [3]:
# The climate-fever dataset is in a format which the transformers Trainer does not understand
# It must be preprocessed using the functions in preprocessing.py

df = pd.read_json("data/climate_fever/climate-fever-dataset-r1.jsonl", lines=True)
preprocessed_df = climate_fever_to_claim_evidence_pairs(df)


In [4]:
# Map evidence_labels to integers so that the Trainer will know what the labels mean
label_dict = {
    "REFUTES": 0,
    "NOT_ENOUGH_INFO": 1,
    "SUPPORTS": 2
}

preprocessed_df["labels"] = preprocessed_df["evidence_label"].map(label_dict)
preprocessed_df

,claim_id,claim,evidence_id,evidence_label,evidence,entropy,labels
0,0,Global warming is driving polar bears toward e...,Extinction risk from global warming:170,NOT_ENOUGH_INFO,"""Recent Research Shows Human Activity Driving ...",0.693147,1
1,0,Global warming is driving polar bears toward e...,Global warming:14,SUPPORTS,Environmental impacts include the extinction o...,0.000000,2
2,0,Global warming is driving polar bears toward e...,Global warming:178,NOT_ENOUGH_INFO,Rising temperatures push bees to their physiol...,0.693147,1
3,0,Global warming is driving polar bears toward e...,Habitat destruction:61,SUPPORTS,"Rising global temperatures, caused by the gree...",0.000000,2
4,0,Global warming is driving polar bears toward e...,Polar bear:1328,NOT_ENOUGH_INFO,"""Bear hunting caught in global warming debate"".",0.693147,1
...,...,...,...,...,...,...,...
7670,3134,"Over the last decade, heatwaves are five times...",Bushfires in Australia:126,SUPPORTS,Australia's climate has warmed by more than on...,0.000000,2
7671,3134,"Over the last decade, heatwaves are five times...",Effects of global warming:86,NOT_ENOUGH_INFO,"In the last 30–40 years, heat waves with high ...",0.693147,1
7672,3134,"Over the last decade, heatwaves are five times...",Global warming:155,NOT_ENOUGH_INFO,Many regions have probably already seen increa...,0.693147,1
7673,3134,"Over the last decade, heatwaves are five times...",Global warming:156,NOT_ENOUGH_INFO,"Since the 1950s, droughts and heat waves have ...",0.693147,1


In [5]:
dataset = Dataset.from_pandas(preprocessed_df)
dataset

Dataset({
    features: ['claim_id', 'claim', 'evidence_id', 'evidence_label', 'evidence', 'entropy', 'labels'],
    num_rows: 7675
})

In [6]:
# Shuffle the dataset! This randomly rearranges the dataset, which is good especially with this one since the same claim appears five times in a row
# The seed parameter means we can access the exact same shuffle again if we need to
dataset = dataset.shuffle(seed=13)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

/home/lukeg/ClimatEnv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# This splits our dataset so that we use 90% of it for training, and 10% for testing
split_dataset = dataset.train_test_split(test_size=0.1)

In [9]:
def custom_tokenize(examples):
    # The code block below this one can be used to find what the max_length should be set to.
    # Otherwise you have too much padding
    # Consider this properly later
    tokenized_output = tokenizer(
        text=[f"Claim: {claim} Evidence: {evidence}" for claim, evidence in zip(examples["claim"], examples["evidence"])],
        max_length=512, 
        padding="max_length", 
        truncation=True)

    return tokenized_output

tokenized_training_dataset = split_dataset["train"].map(custom_tokenize, batched=True)
tokenized_testing_dataset = split_dataset["test"].map(custom_tokenize, batched=True)

Map:   0%|          | 0/6907 [00:00<?, ? examples/s]

Map:   0%|          | 0/768 [00:00<?, ? examples/s]

In [10]:

# Just for viewing purposes. Input_ids are the tokens, and attention_masks are whether they represent actual words or not.
# The max_length is set to 512 so every entry has been padded to be this long, which may be unnecessary
print(tokenized_training_dataset[6]["claim"])
print(tokenized_training_dataset[6]["evidence"])
print(tokenized_training_dataset[6]["labels"])
print(tokenized_training_dataset[6]["input_ids"])
print(tokenized_training_dataset[6]["attention_mask"])

print(tokenized_testing_dataset[2]["claim"])
print(tokenized_testing_dataset[2]["evidence"])
print(tokenized_training_dataset[2]["labels"])
print(tokenized_testing_dataset[2]["input_ids"])
print(tokenized_testing_dataset[2]["attention_mask"])

HFCs (hydrofluorocarbons) are tens of thousands of times more polluting than carbon dioxide.
They do not harm the ozone layer as much as the compounds they replace, but they do contribute to global warming, with thousands of times the warming potential of carbon dioxide.
2
[0, 45699, 35, 289, 5268, 29, 36, 30420, 1001, 33266, 368, 1975, 271, 16830, 43, 32, 7281, 9, 1583, 9, 498, 55, 2902, 15024, 87, 4363, 1437, 50346, 4, 27956, 35, 252, 109, 45, 4798, 5, 34011, 10490, 25, 203, 25, 5, 18291, 51, 3190, 6, 53, 51, 109, 1437, 50498, 7, 720, 8232, 6, 19, 1583, 9, 498, 5, 8232, 1437, 50273, 9, 4363, 1437, 50346, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [11]:
# Remove ignore_mismatched_sizes when needed - this replaces the head of the pretrained model (because if using
# climateBERT/environmental-claims, it has already been fine tuned and has 2 labels
model = AutoModelForSequenceClassification.from_pretrained(
    "climatebert/distilroberta-base-climate-f",
    num_labels=3,
    # ignore_mismatched_sizes=True
).to("cuda")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
print(next(model.parameters()).device)

cuda:0


In [13]:
model.gradient_checkpointing_enable()

In [14]:
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

In [ ]:
# This is where we set the hyperparameters
training_args = TrainingArguments(
    output_dir="./results/climateBERT-base/climate_fever/second_run",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=600,
    logging_strategy="steps",
    logging_steps=200,
    save_strategy="no",
    save_steps=500,
    fp16=True,                          # Use 16-bit floating point instead of 32 - makes computation faster
    warmup_ratio=0.05,                    # Allows the model to adapt a little
    # gradient_accumulation_steps=2       # Might help with OOM errors, if we have them
    learning_rate=2e-5,
    push_to_hub=False,
)

In [16]:
trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_dataset,
    tokenizer=tokenizer,
    eval_dataset=tokenized_testing_dataset,
    compute_metrics=calculate_metrics,
)

In [17]:
trainer.train()

  0%|          | 0/2592 [00:00<?, ?it/s]

{'loss': 0.9405, 'grad_norm': 4.262098789215088, 'learning_rate': 1.9439480097481723e-05, 'epoch': 0.23}
{'loss': 0.844, 'grad_norm': 6.092226028442383, 'learning_rate': 1.7814784727863527e-05, 'epoch': 0.46}
{'loss': 0.786, 'grad_norm': 11.764533996582031, 'learning_rate': 1.619008935824533e-05, 'epoch': 0.69}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.767102062702179, 'eval_accuracy': 0.67578125, 'eval_f1_score': 0.6039256104681285, 'eval_runtime': 9.0085, 'eval_samples_per_second': 85.252, 'eval_steps_per_second': 10.657, 'epoch': 0.69}
{'loss': 0.7447, 'grad_norm': 7.9496541023254395, 'learning_rate': 1.4573517465475225e-05, 'epoch': 0.93}
{'loss': 0.673, 'grad_norm': 13.883233070373535, 'learning_rate': 1.2948822095857027e-05, 'epoch': 1.16}
{'loss': 0.6621, 'grad_norm': 19.317888259887695, 'learning_rate': 1.1332250203086923e-05, 'epoch': 1.39}


  0%|          | 0/96 [00:00<?, ?it/s]

...

  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.8482264876365662, 'eval_accuracy': 0.6497395833333334, 'eval_f1_score': 0.6572631989717184, 'eval_runtime': 8.8812, 'eval_samples_per_second': 86.475, 'eval_steps_per_second': 10.809, 'epoch': 2.08}
{'loss': 0.4986, 'grad_norm': 9.661898612976074, 'learning_rate': 4.833468724614135e-06, 'epoch': 2.31}
{'loss': 0.4658, 'grad_norm': 23.311182022094727, 'learning_rate': 3.2087733549959386e-06, 'epoch': 2.55}
{'loss': 0.4456, 'grad_norm': 50.64796829223633, 'learning_rate': 1.5840779853777418e-06, 'epoch': 2.78}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.789134681224823, 'eval_accuracy': 0.7044270833333334, 'eval_f1_score': 0.7076457958805147, 'eval_runtime': 8.7581, 'eval_samples_per_second': 87.69, 'eval_steps_per_second': 10.961, 'epoch': 2.78}
{'train_runtime': 1056.7056, 'train_samples_per_second': 19.609, 'train_steps_per_second': 2.453, 'train_loss': 0.6353937814265121, 'epoch': 3.0}


TrainOutput(global_step=2592, training_loss=0.6353937814265121, metrics={'train_runtime': 1056.7056, 'train_samples_per_second': 19.609, 'train_steps_per_second': 2.453, 'total_flos': 2744905918178304.0, 'train_loss': 0.6353937814265121, 'epoch': 3.0})

In [18]:
# trainer.save_model("./results/climateBERT-base/climate_fever/first_run")
# Please remember to delete model.safetensors BEFORE adding to git. Causes issues...
# Also it is probably not worth running this block until the model is worth keeping

In [19]:
# Metrics are not included in the save model so we need to save them separately
metrics = trainer.evaluate()

  0%|          | 0/96 [00:00<?, ?it/s]

In [20]:
with open("./results/climateBERT-base/climate_fever/second_run/eval_metrics.json", "w") as output_file:
    json.dump(metrics, output_file)